In [3]:
import pandas as pd
from connection import connect


In [6]:
#connection
SCHEMA='production'
co_oltp,etl_conn=connect(SCHEMA)


['product_list_price_history', 'product_model', 'product_model_illustration', 'product_model_prod_desc_culture', 'product_subcategory', 'scrap_reason', 'transaction_history', 'transaction_history_archive', 'bill_of_materials', 'unit_measure', 'document', 'work_order', 'culture', 'illustration', 'location', 'product', 'product_category', 'product_cost_history', 'product_photo', 'product_product_photo', 'product_review', 'product_description', 'work_order_routing', 'product_document', 'product_inventory']


## Extract

In [7]:
#extract DimProductCategory


from deep_translator import GoogleTranslator

category=pd.read_sql_table('product_category',co_oltp,schema='production')
category


,product_category_id,name,rowguid,modified_date
0,1,Bikes,CFBDA25C-DF71-47A7-B81B-64EE161AA37C,2008-04-30 00:00:00+00:00
1,2,Components,C657828D-D808-4ABA-91A3-AF2CE02300E9,2008-04-30 00:00:00+00:00
2,3,Clothing,10A7C342-CA82-48D4-8A38-46A2EB089B74,2008-04-30 00:00:00+00:00
3,4,Accessories,2BE3BE36-D9A2-4EEE-B593-ED895D97C2A6,2008-04-30 00:00:00+00:00


## Transform


In [8]:
category=category.rename(columns={'product_category_id':'product_category_alternate_key','name':'english_product_category_name'})

category=category.drop(['rowguid','modified_date'],axis=1)
category['spanish_product_category_name']=category['english_product_category_name'].apply(lambda x : GoogleTranslator(source='en',target='es').translate(x) if x is not None else None )
category['french_product_category_name']=category['english_product_category_name'].apply(lambda x : GoogleTranslator(source='en',target='fr').translate(x) if x is not None else None )

category

,product_category_alternate_key,english_product_category_name,spanish_product_category_name,french_product_category_name
0,1,Bikes,bicicletas,Vélos
1,2,Components,Componentes,Composants
2,3,Clothing,Ropa,Vêtements
3,4,Accessories,Accesorios,Accessoires


## Load

In [6]:
category.to_sql('dim_product_category', etl_conn, if_exists='append',index=False)

4